In [7]:
import pandas as pd

# 导入数据
file = open('导入订单utf8.csv', 'r', encoding='utf-8')
df = pd.read_csv(file, header=0, sep=',', encoding='utf-8', engine='python')

# 添加年份和月份列
df['发货日期'] = pd.to_datetime(df['发货日期'])
df['年份'] = df['发货日期'].dt.year
df['月份'] = df['发货日期'].dt.month

# 将销售额按价格带进行分组
price_bins = [0, 100, 500, 1000, 2000, 10000]
price_labels = ['0-100元', '100-500元', '500-1000元', '1000-2000元', '2000-10000元']
df['价格带'] = pd.cut(df['销售额'], bins=price_bins, labels=price_labels)

# 按年份、月份和价格带分组，计算销售额和折扣
grouped = df.groupby(['年份', '月份', '价格带'])[['销售额', '数量', '折扣']].sum()
grouped['平均折扣'] = grouped['折扣'] / grouped['数量']

# 按年份和月份分组，计算每个月份每个价格带的销售额和平均折扣
result = {}
for year in df['年份'].unique():
    year_data = grouped.loc[year]
    for month in df['月份'].unique():
        month_data = year_data.loc[month]
        for label in price_labels:
            label_data = month_data.loc[label]
            if label not in result:
                result[label] = [(year, month, label_data['销售额'], label_data['平均折扣'])]
            else:
                result[label].append((year, month, label_data['销售额'], label_data['平均折扣']))

# 输出结果
print('每年每个月的价格带销售额和平均折扣：')
for label, data in result.items():
    print(label)
    for item in data:
        print('年份：{}，月份：{}，销售额：{}，平均折扣：{}'.format(item[0], item[1], item[2], item[3]))

每年每个月的价格带销售额和平均折扣：
0-100元
年份：2018，月份：4，销售额：984.8720000000001，平均折扣：0.12941176470588237
年份：2018，月份：6，销售额：1434.5240000000001，平均折扣：0.0975609756097561
年份：2018，月份：12，销售额：2169.16，平均折扣：0.07941176470588236
年份：2018，月份：10，销售额：2352.084，平均折扣：0.14426229508196722
年份：2018，月份：11，销售额：1680.588，平均折扣：0.12830188679245286
年份：2018，月份：3，销售额：964.04，平均折扣：0.09230769230769233
年份：2018，月份：9，销售额：944.048，平均折扣：0.05185185185185186
年份：2018，月份：7，销售额：948.6120000000001，平均折扣：0.0903225806451613
年份：2018，月份：8，销售额：1846.852，平均折扣：0.13437500000000002
年份：2018，月份：5，销售额：2191.868，平均折扣：0.15774647887323945
年份：2018，月份：1，销售额：1068.564，平均折扣：0.06666666666666667
年份：2018，月份：2，销售额：941.444，平均折扣：0.06666666666666667
年份：2017，月份：4，销售额：635.208，平均折扣：0.14545454545454548
年份：2017，月份：6，销售额：1276.632，平均折扣：0.09142857142857143
年份：2017，月份：12，销售额：940.184，平均折扣：0.06153846153846154
年份：2017，月份：10，销售额：1347.192，平均折扣：0.12631578947368424
年份：2017，月份：11，销售额：2408.42，平均折扣：0.12054794520547946
年份：2017，月份：3，销售额：746.172，平均折扣：0.08750000000000001
年份：2017，月份：9，销售额：979.048，平均折扣：0.1

In [8]:
# 计算每个月份每个价格带的销售额占比
grouped['销售额占比'] = grouped.groupby(['年份', '月份'])['销售额'].apply(lambda x: x / x.sum())

# 计算2018年每个月份高价位销售占比和前一个月份的占比，判断是否出现下滑
result = {}
for month in df[df['年份'] == 2018]['月份'].unique():
    high_price_data = grouped.loc[(2018, month, '2000-10000元')]
    if month == 1:
        last_month_data = grouped.loc[(2017, 12, '2000-10000元')]
    else:
        last_month_data = grouped.loc[(2018, month-1, '2000-10000元')]
    if high_price_data['销售额占比'] < last_month_data['销售额占比']:
        result[month] = high_price_data['销售额占比'] / last_month_data['销售额占比']

# 输出结果
if len(result) == 0:
    print('2018年没有月份的高价位销售占比出现下滑')
else:
    print('2018年下列月份的高价位销售占比出现了下滑：')
    for month, ratio in result.items():
        print('月份：{}，高价位销售占比下滑比例：{:.2f}%'.format(month, (1-ratio)*100))


2018年下列月份的高价位销售占比出现了下滑：
月份：6，高价位销售占比下滑比例：4.43%
月份：12，高价位销售占比下滑比例：5.06%
月份：11，高价位销售占比下滑比例：3.95%
月份：7，高价位销售占比下滑比例：3.10%
月份：3，高价位销售占比下滑比例：5.26%


C:\Users\01\AppData\Local\Temp\ipykernel_27376\382831527.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  grouped['销售额占比'] = grouped.groupby(['年份', '月份'])['销售额'].apply(lambda x: x / x.sum())
